In [17]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [18]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [19]:
s = Structure(test_datas[6])
print(s.display())

solver = BasicSolver(s)
# solver.idxes_need_to_solve

+-----+-----+-----+
|6 . 7|5 . 2|9 . .|
|     |     |     |
|. . 2|9 . 4|6 . 7|
|     |     |     |
|1 . 9|7 6 8|3 2 .|
+-----+-----+-----+
|. 9 .|1 . .|. 6 .|
|     |     |     |
|. . .|6 9 5|. 7 1|
|     |     |     |
|. 1 6|2 . .|. 9 .|
+-----+-----+-----+
|9 6 5|4 7 1|8 3 2|
|     |     |     |
|8 . 1|3 . 9|7 . 6|
|     |     |     |
|. 7 .|8 . 6|1 . 9|
+-----+-----+-----+



In [20]:
# solver.check_idx_only(62)
# print(solver.ready)
# solver.check_idx_last_left(62)
# print(solver.ready)
print(solver.check_scanned_drop('5'))
print(solver.check_area_drop('5'))
solver.ready

+-----+-----+-----+
|6   7|5   2|9    |
|     |     |     |
|. . 2|9   4|6 . 7|
|     |     |     |
|1 . 9|7 6 8|3 2 .|
+-----+-----+-----+
|. 9  |1    |. 6 .|
|     |     |     |
|     |6 9 5|  7 1|
|     |     |     |
|. 1 6|2    |. 9 .|
+-----+-----+-----+
|9 6 5|4 7 1|8 3 2|
|     |     |     |
|8   1|3 . 9|7 . 6|
|     |     |     |
|  7  |8 . 6|1 . 9|
+-----+-----+-----+

False
+-----+-----+-----+
|6   7|5   2|9    |
|     |     |     |
|  . 2|9   4|6   7|
|     |     |     |
|1 . 9|7 6 8|3 2 .|
+-----+-----+-----+
|  9  |1    |. 6 .|
|     |     |     |
|     |6 9 5|  7 1|
|     |     |     |
|  1 6|2    |. 9 .|
+-----+-----+-----+
|9 6 5|4 7 1|8 3 2|
|     |     |     |
|8   1|3   9|7   6|
|     |     |     |
|  7  |8   6|1   9|
+-----+-----+-----+

True


[(10, '5'), (26, '5')]

In [15]:
tmp = "	6	 	7	5	 	2	9	 	 \
	 	 	2	9	 	4	6	 	7\
	1	 	9	7	6	8	3	2	 \
	 	9	 	1	 	 	 	6	 \
	 	 	 	6	9	5	 	7	1\
	 	1	6	2	 	 	 	9	 \
	9	6	5	4	7	1	8	3	2\
	8	 	1	3	 	9	7	 	6\
	 	7	 	8	 	6	1	 	9"

In [16]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

161


'6,.,7,5,.,2,9,.,.,.,.,2,9,.,4,6,.,7,1,.,9,7,6,8,3,2,.,.,9,.,1,.,.,.,6,.,.,.,.,6,9,5,.,7,1,.,1,6,2,.,.,.,9,.,9,6,5,4,7,1,8,3,2,8,.,1,3,.,9,7,.,6,.,7,.,8,.,6,1,.,9'